# Initialize

In [8]:
import os
print(os.getenv("PYTHONSTARTUP"))

# TODO https://panel.holoviz.org/user_guide/Pipelines.html

# Options

## Debugging level
debug_level = 0

## Anonymize toggle
anonymize = False


/Users/miladnazar/Desktop/Homework/GroupProject_Project_8V2/Code/lib2/imports.py


In [9]:
# Initialize framework
constants, tools = init_preprocessing(debug_level)
%matplotlib inline


In [10]:
# Build the anonymous customer name mapping

## Read the datasets into Pandas DataFrames
atlas               = pd.read_csv(constants.RAW_ATLAS_FILE_PATH, index_col="Customers")
forecast            = pd.read_csv(constants.RAW_FORECAST_DATA_FILE_PATH, index_col="Organization Name")
revenue2020         = pd.read_csv(constants.RAW_REVENUE2020_FILE_PATH, index_col="Name")
revenue2020A        = pd.read_csv(constants.RAW_REVENUE2020A_FILE_PATH, index_col="Payee Name")
potential_customers_file    = pd.read_csv(constants.RAW_POTENTIAL_CUSTOMERS_FILE_PATH, index_col="Customer Name")

## Extract paying customers
atlas_customers_raw             = atlas.index.to_list()
forecast_customers_raw          = forecast.index.to_list()
revenue2020_customers_raw       = revenue2020.index.to_list()
revenue2020A_customers_raw      = revenue2020A.index.to_list()
potential_customers_raw         = potential_customers_file.index.to_list()

## Clean up names
atlas_customers_cleaned             = tools.tool_customer.cleanup_customer_names(atlas_customers_raw)
forecast_customers_cleaned          = tools.tool_customer.cleanup_customer_names(forecast_customers_raw)
revenue2020_customers_cleaned       = tools.tool_customer.cleanup_customer_names(revenue2020_customers_raw)
revenue2020A_customers_cleaned      = tools.tool_customer.cleanup_customer_names(revenue2020A_customers_raw)
potential_customers_cleaned         = tools.tool_customer.cleanup_customer_names(potential_customers_raw)

## Build name mapping
name_mapping                            = tools.tool_mapping.build_name_mapping(atlas_customers_cleaned, {})
name_mapping                            = tools.tool_mapping.build_name_mapping(forecast_customers_cleaned, name_mapping)
name_mapping                            = tools.tool_mapping.build_name_mapping(revenue2020_customers_cleaned, name_mapping)
name_mapping                            = tools.tool_mapping.build_name_mapping(revenue2020A_customers_cleaned, name_mapping)
name_mapping_potential_customers        = tools.tool_mapping.build_name_mapping(potential_customers_cleaned, {})

## Export the name mappings
tools.tool_mapping.write_customer_name_mapping(name_mapping)
tools.tool_mapping.write_lookup_table(name_mapping_potential_customers, constants.LUT_POTENTIAL_CUSTOMER_NAME_FILE_PATH)

## Export the customer name list
if anonymize:
    tools.tool_mapping.write_list( list(set(name_mapping.values())),
        constants.LIST_CUSTOMER_NAME_LIST_FILE_PATH)
    tools.tool_mapping.write_list( list(set(name_mapping_potential_customers.values())),
        constants.LIST_POTENTIAL_CUSTOMER_NAME_LIST_FILE_PATH)
else:
    tools.tool_mapping.write_list( list(set(name_mapping.keys())),
        constants.LIST_CUSTOMER_NAME_LIST_FILE_PATH)
    tools.tool_mapping.write_list( list(set(name_mapping_potential_customers.keys())),
        constants.LIST_POTENTIAL_CUSTOMER_NAME_LIST_FILE_PATH)


In [11]:
# Anonymize the raw data files

## Clean up names
atlas_customers_cleaned         = tools.tool_customer.cleanup_customer_names(atlas_customers_raw)
forecast_customers_cleaned      = tools.tool_customer.cleanup_customer_names(forecast_customers_raw)
revenue2020_customers_cleaned   = tools.tool_customer.cleanup_customer_names(revenue2020_customers_raw)
revenue2020A_customers_cleaned  = tools.tool_customer.cleanup_customer_names(revenue2020A_customers_raw)


if anonymize:

    print("Anonymizing data!")

    ## Import name mapping
    customer_name_mapping = tool_mapping.read_customer_name_mapping()

    ## Map the customer names
    atlas_customers_finalized              = tools.tool_customer.anonymize_customer_list(atlas_customers_cleaned, customer_name_mapping)
    forecast_customers_finalized           = tools.tool_customer.anonymize_customer_list(forecast_customers_cleaned, customer_name_mapping)
    revenue2020_customers_finalized        = tools.tool_customer.anonymize_customer_list(revenue2020_customers_cleaned, customer_name_mapping)
    revenue2020A_customers_finalized       = tools.tool_customer.anonymize_customer_list(revenue2020A_customers_cleaned, customer_name_mapping)


else:
    
    print("NOT anonymizing data!")
    atlas_customers_finalized = atlas_customers_cleaned
    forecast_customers_finalized = forecast_customers_cleaned
    revenue2020_customers_finalized = revenue2020_customers_cleaned
    revenue2020A_customers_finalized = revenue2020A_customers_cleaned


# Fix the customers in the DataFrames
atlas           = tools.tool_customer.set_customers_index(atlas, atlas_customers_finalized, "Customers")
forecast        = tools.tool_customer.set_customers_index(forecast, forecast_customers_finalized, "Organization Name")
revenue2020     = tools.tool_customer.set_customers_index(revenue2020, revenue2020_customers_finalized, "Name")
revenue2020A    = tools.tool_customer.set_customers_index(revenue2020A, revenue2020A_customers_finalized, "Payee Name")

NOT anonymizing data!


In [12]:
# Export
atlas.to_csv(constants.ANON_ATLAS_FILE_PATH)
forecast.to_csv(constants.ANON_FORECAST_DATA_FILE_PATH)
revenue2020.to_csv(constants.ANON_REVENUE2020_FILE_PATH)
revenue2020A.to_csv(constants.ANON_REVENUE2020A_FILE_PATH)


In [13]:
atlas

,Number of Users,Invoice Date,Invoice #,Invoice Amount,Subscription,Account Code,Dates of service,Address,State,Lat,Long
Customers,,,,,,,,,,,
Alabama A&M University-College Of Education,NaN,3/23/2018,105,"$5,460.00",3 Years,4700-0-00-00000-00-0000,3 Years Subscription 3/21/18 to 3/31/21,"4900 Meridian St N, Huntsville, AL 35811",AL,34.785061,-86.573280
Albertus Magnus College,NaN,5/16/2018,127,"$5,670.00",3 Years,4700-0-00-00000-00-0000,3 Years Subscription 9/1/18 to 8/31/21,"700 Prospect St, New Haven, CT 06511",CT,41.331299,-72.921440
Albuquerque Public Schools,NaN,11/20/2018,161,"$2,270.00",1 Year,4700-0-00-00000-00-0000,1 Year Subscription 11/19/18 to 11/30/19,"Po Box 25704 Albuquerque, NM 87125",NM,35.092410,-106.642950
Alcorn State University,NaN,10/31/2017,NaN,"$1,700.00",1 Year,4700-0-00-00000-00-0000,1 Year Subscription 8/31/17 to 8/31/18,"1000 Asu Dr, Lorman, MS 39096",MS,31.875710,-91.141739
Alderson Broaddus University,50.0,10/7/2020,NB100820K,$500.00,3 Months,4700-0-00-00000-00-0000,3 Months Subscription 10/2/2020 to 12/31/2020,"101 College Hill Rd, Philippi, WV 26416",WV,39.160261,-80.049048
...,...,...,...,...,...,...,...,...,...,...,...
Winthrop University,100.0,9/23/2019,222,"$1,000.00",1 Year,4700-0-00-00000-00-0000,1 Year Subscription 9/20/19 to 9/30/20,"701 Oakland Ave, Rock Hill, SC 29730",SC,34.938466,-81.029872
Winthrop University,100.0,10/13/2020,NB101320C,"$1,000.00",1 Year,4700-0-00-00000-00-0000,1 Year Subscription 9/30/2020 to 9/30/2021,"701 Oakland Ave, Rock Hill, SC 29730",SC,34.938466,-81.029872
Yakima Valley College,56.0,9/21/2020,NB092120A,"$1,030.00",6 Months,4700-0-00-00000-00-0000,6 Months Subscription 9/18/2020 to 03/18/2021,"PO Box 22520, Yakima, WA 99807-2520",WA,-35.001338,117.875053
